<a href="https://colab.research.google.com/github/caesarcc/python-tcc-url-fakenews-check/blob/main/jupyter/passo05_preditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q lime

     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 54.2 MB/s 
     |████████████████████████████████| 6.6 MB 43.8 MB/s 
     |████████████████████████████████| 84 kB 1.2 MB/s 
     |████████████████████████████████| 275 kB 5.1 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import hashlib
import torch
from lime.lime_text import LimeTextExplainer
from scipy.special import softmax
from transformers import BertForSequenceClassification, BertTokenizer
from transformers.file_utils import is_torch_available

In [76]:
MODELO_BERT = "/content/drive/MyDrive/PUC/TCC/modelos/bertimbau_avaliar_noticias"
model = BertForSequenceClassification.from_pretrained(MODELO_BERT, num_labels=2, output_attentions=False, output_hidden_states=False)
tokenizer = BertTokenizer.from_pretrained(MODELO_BERT, do_lower_case=False)

Preditor deve antes de rodar modelo<br>
> Limpar texto<br>
> Remover stopwords<br>
> Lematizar<br>
-- Caso o texto seja maior que 400 palavras<br>
> Sumarizar texto (sem lematizaÃ§Ã£o e com stopwords)

In [5]:
def qual_device():
    if is_torch_available() and torch.cuda.is_available():
        return "cuda"
    return "cpu"
qual_device()

'cuda'

In [77]:
model.to(qual_device())
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [74]:
def explicar(texto, classes):
    def prever_explicando(texto):
        temp = pd.DataFrame(texto, columns = ['texto'])
        temp['tokens'] = temp['texto'].apply(lambda x: np.array(tokenizer.encode(x, add_special_tokens=True, max_length=400, padding='max_length', truncation=True)))
        valores = torch.tensor(np.array(temp['tokens'].values.tolist())).to(torch.int64)
        valores = valores.to(qual_device())
        resultados = []
        for valor in valores:
            with torch.no_grad():
                outputs = model(valor.unsqueeze(0), token_type_ids=None)
            logits = outputs.logits.cpu().detach().numpy()
            logits = softmax(logits)
            resultados.append(logits[0])
        retorno = np.array(resultados)
        return retorno    
    explainer = LimeTextExplainer(class_names=classes)
    exp = explainer.explain_instance(texto, prever_explicando)
    exp.save_to_file(f'{hashlib.md5(texto.encode()).hexdigest()}.html')
    return exp.as_list()

In [85]:
def prever(texto, classes):
    tokens = tokenizer.encode(texto, padding='max_length', truncation=True, max_length=400)
    valores = torch.tensor(tokens).to(qual_device())
    with torch.no_grad():
        outputs = model(valores.unsqueeze(0), token_type_ids=None)
    classes_previstas = outputs.logits.detach().cpu().numpy()
    predicao = np.argmax(classes_previstas, axis=1).flatten()
    return softmax(classes_previstas), predicao

In [105]:
import torch.nn.functional as F
def prever_novo(texto, classes):
    encoded_review = tokenizer.encode_plus(
        texto,
        max_length=400,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
      )

    input_ids = encoded_review['input_ids'].to(qual_device())
    attention_mask = encoded_review['attention_mask'].to(qual_device())

    output = model(input_ids, attention_mask).logits
    _, prediction = torch.max(output, dim=1)
    probs = F.softmax(output, dim=1)

    print(pd.DataFrame(probs.tolist()[0], classes)[0])


In [106]:
classes = ['Confiável','Falso']
texto = """O popular apresentador Ratinho foi ví­tima de um acidente de carro com toda a sua família em São Paulo, 
e infelizmente não resistiu aos ferimentos e morreu"""
prever_novo(texto, classes)

Confiável    0.001938
Falso        0.998062
Name: 0, dtype: float64


In [110]:
texto = """Dicionário Oscar : categoria técnico premir ? . Conheça prêmios Melhor fotografia , edição design produção – glamurosos , 
responsável reconhecer centena profissional trabalhar trás câmeras."""
prever_novo(texto, classes)

Confiável    0.97017
Falso        0.02983
Name: 0, dtype: float64


In [94]:
tokens = tokenizer.encode(texto, padding='max_length', truncation=True, max_length=400)
valores = torch.tensor(tokens).to(qual_device())
valores.unsqueeze(0)

tensor([[  101,   200, 16484, 18868, 18199, 22317,   118,  5857,  1564,   790,
           333, 11407, 22278,   171, 14081,   117,   123,  7347, 22278,   461,
         22303,  2916, 13553,   113,   267, 22317,   114,  6454,   260,  4739,
          3290,   221, 13142, 22282,   146,  1640,  7435, 18286,   119, 16663,
          4820,   123,  6698,   171,  1640,   117,   179,  2471, 17293,  1555,
           125, 14651,   110,   180,  1521,   117,   123,  7347, 22278,  2694,
           179, 18286,   346,  4737,   176, 12439, 22282,   298,  3955, 11720,
         11735, 22281,   240,   347,  1161,   119,   107, 13716,   259,   126,
          1913,   501, 19673, 22281,   180,  3338,   146, 18286,  4737, 20327,
          8807,   119,   989,   146,  3901,   437, 14151,   181,   320,  3811,
           119,  7178, 11989,   243,   107,   117,  3415,   119,   231,  3101,
           125,   989,   478,   171, 14081,  3368, 11407,   159,   461, 22303,
          2916, 13553,   229,  6151,   118, 14258,  

In [89]:
prever(texto, classes)

(array([[0.06074722, 0.9392526 ]], dtype=float32), array([1]))

In [90]:
explicar(texto, classes)    

[('Brasil', 0.15024944879834823),
 ('essas', 0.13739181909935577),
 ('portuguesa', 0.1356705543994631),
 ('pela', 0.0836190754914776),
 ('muito', 0.08183565436870122),
 ('vão', 0.07911594281843873),
 ('e', 0.07872064933299759),
 ('similaridades', -0.06427648529743125),
 ('histórico', -0.03143715967272221),
 ('comum', -0.02742048612767332)]